<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EB%AC%BC%EB%A5%98%20%EC%9C%A0%ED%86%B5%EB%9F%89%20%EC%98%88%EC%B8%A1%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EB%AC%BC%EB%A5%98_%EC%9C%A0%ED%86%B5%EB%9F%89_%EC%98%88%EC%B8%A1_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

https://dacon.io/competitions/official/235867/codeshare/3888?page=1&dtype=recent

In [2]:
train=pd.read_csv('/content/drive/MyDrive/물류 유통량 예측 경진대회/train.csv')
test=pd.read_csv('/content/drive/MyDrive/물류 유통량 예측 경진대회/test.csv')
sample=pd.read_csv('/content/drive/MyDrive/물류 유통량 예측 경진대회/sample_submission.csv')

In [3]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [4]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


# EDA

In [6]:
train.columns

Index(['index', '송하인_격자공간고유번호', '수하인_격자공간고유번호', '물품_카테고리', '운송장_건수'], dtype='object')

In [7]:
train['송하인_격자공간고유번호'].value_counts()

5011000078068400    3882
5013000610049100    1790
5013000731055200    1235
5011000137030100    1137
5013000821028200     731
                    ... 
4511300030055300       1
4719000321023400       1
4423000423020300       1
4711300748080100       1
2811000139076100       1
Name: 송하인_격자공간고유번호, Length: 4229, dtype: int64

In [8]:
train['수하인_격자공간고유번호'].value_counts()

5013000635005300    35
5011000543041100    33
5011000314069300    33
5011000544072300    30
5011000318044100    30
                    ..
5011000109040100     1
4183000550093100     1
4425000028078200     1
1147000011064300     1
5011000263065200     1
Name: 수하인_격자공간고유번호, Length: 26875, dtype: int64

https://www.bigdata-region.kr/#/dataset/7193fc33-73c6-4dcf-b06b-534f4998cb72
여기 자료를 보니 시군구코드가 앞의 5자리 숫자인가보다. 

송하인 격자공간고유번호는 1 ~ 5, 6 ~ 9, 10, 11 ~ 16자릿수로 나누고 수하인 격자공간고유번호는 자릿수 별로 변수를 생성하였습니다.

총 22개의 설명변수로 이루어진 데이터로 변환하였습니다.

In [3]:
def numround(number, digit):
  num=[]
  while(number!=0):
    num.append(number % 10)
    number = number //10

  return int(num[-digit])

In [4]:
from tqdm import tqdm

In [5]:
for i in tqdm(range(16)):
  train[f'SEND_SPG_INNB_{i+1}'] = 0
  train[f'REC_SPG_INNB_{i+1}'] = 0
  test[f'SEND_SPG_INNB_{i+1}'] = 0
  test[f'REC_SPG_INNB_{i+1}'] = 0
  for j in range(train.shape[0]):
    train.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(train.loc[j,'송하인_격자공간고유번호'],i+1)
    train.loc[j,f'REC_SPG_INNB_{i+1}']=numround(train.loc[j,'수하인_격자공간고유번호'],i+1)

  for j in range(test.shape[0]):
    test.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(test.loc[j,'송하인_격자공간고유번호'],i+1)
    test.loc[j,f'REC_SPG_INNB_{i+1}']=numround(test.loc[j,'수하인_격자공간고유번호'],i+1)

100%|██████████| 16/16 [08:01<00:00, 30.09s/it]


In [6]:
train['SEND_SPG_INNB_1~5']=train['SEND_SPG_INNB_1']+train['SEND_SPG_INNB_2']+train['SEND_SPG_INNB_3']+train['SEND_SPG_INNB_4']+train['SEND_SPG_INNB_5']
train['SEND_SPG_INNB_6~9']=train['SEND_SPG_INNB_6']+train['SEND_SPG_INNB_7']+train['SEND_SPG_INNB_8']+train['SEND_SPG_INNB_9']
train['SEND_SPG_INNB_10']=train['SEND_SPG_INNB_10']
train['SEND_SPG_INNB_11~16']=train['SEND_SPG_INNB_11']+train['SEND_SPG_INNB_12']+train['SEND_SPG_INNB_13']+train['SEND_SPG_INNB_14']+train['SEND_SPG_INNB_15']+train['SEND_SPG_INNB_16']

test['SEND_SPG_INNB_1~5']=test['SEND_SPG_INNB_1']+test['SEND_SPG_INNB_2']+test['SEND_SPG_INNB_3']+test['SEND_SPG_INNB_4']+test['SEND_SPG_INNB_5']
test['SEND_SPG_INNB_6~9']=test['SEND_SPG_INNB_6']+test['SEND_SPG_INNB_7']+test['SEND_SPG_INNB_8']+test['SEND_SPG_INNB_9']
test['SEND_SPG_INNB_10']=test['SEND_SPG_INNB_10']
test['SEND_SPG_INNB_11~16']=test['SEND_SPG_INNB_11']+test['SEND_SPG_INNB_12']+test['SEND_SPG_INNB_13']+test['SEND_SPG_INNB_14']+test['SEND_SPG_INNB_15']+test['SEND_SPG_INNB_16']

In [7]:
train.index=train['index']
test.index=test['index']
train.drop(['수하인_격자공간고유번호','송하인_격자공간고유번호','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)
test.drop(['수하인_격자공간고유번호','송하인_격자공간고유번호','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)

In [8]:
for col in test.columns:
  train[col]=train[col].astype('category')
  test[col]=test[col].astype('category')

In [9]:
train.head()

,물품_카테고리,운송장_건수,REC_SPG_INNB_1,REC_SPG_INNB_2,REC_SPG_INNB_3,REC_SPG_INNB_4,REC_SPG_INNB_5,REC_SPG_INNB_6,REC_SPG_INNB_7,REC_SPG_INNB_8,REC_SPG_INNB_9,SEND_SPG_INNB_10,REC_SPG_INNB_10,REC_SPG_INNB_11,REC_SPG_INNB_12,REC_SPG_INNB_13,REC_SPG_INNB_14,REC_SPG_INNB_15,REC_SPG_INNB_16,SEND_SPG_INNB_1~5,SEND_SPG_INNB_6~9,SEND_SPG_INNB_11~16
index,,,,,,,,,,,,,,,,,,,,,,
0,음반,3,2,8,7,1,0,0,0,1,9,5,2,0,6,9,3,0,0,7,14,11
1,문화컨텐츠,3,5,0,1,1,0,0,0,2,6,0,4,0,2,4,4,0,0,17,15,10
2,농산물,3,1,1,2,0,0,0,0,0,0,8,7,0,0,5,4,0,0,7,7,18
3,기타식품,7,5,0,1,1,0,0,0,5,8,8,7,0,1,9,4,0,0,15,4,10
4,농산물,3,2,8,2,3,7,0,0,0,1,8,0,0,7,6,3,0,0,7,7,18


In [12]:
col=train['물품_카테고리'].value_counts()
col

농산물              20321
문화컨텐츠             1091
음료                1034
수산                 860
가공식품               846
                 ...  
태블릿PC/노트북액세서리        4
기타출산/육아              4
선케어                  4
유아가구                 4
스포츠잡화                4
Name: 물품_카테고리, Length: 100, dtype: int64

In [13]:
train['물품_카테고리'].value_counts()

농산물              20321
문화컨텐츠             1091
음료                1034
수산                 860
가공식품               846
                 ...  
태블릿PC/노트북액세서리        4
기타출산/육아              4
선케어                  4
유아가구                 4
스포츠잡화                4
Name: 물품_카테고리, Length: 100, dtype: int64

In [10]:
train['물품_카테고리']=train['물품_카테고리'].astype('category')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   물품_카테고리              31684 non-null  category
 1   운송장_건수               31684 non-null  int64   
 2   REC_SPG_INNB_1       31684 non-null  category
 3   REC_SPG_INNB_2       31684 non-null  category
 4   REC_SPG_INNB_3       31684 non-null  category
 5   REC_SPG_INNB_4       31684 non-null  category
 6   REC_SPG_INNB_5       31684 non-null  category
 7   REC_SPG_INNB_6       31684 non-null  category
 8   REC_SPG_INNB_7       31684 non-null  category
 9   REC_SPG_INNB_8       31684 non-null  category
 10  REC_SPG_INNB_9       31684 non-null  category
 11  SEND_SPG_INNB_10     31684 non-null  category
 12  REC_SPG_INNB_10      31684 non-null  category
 13  REC_SPG_INNB_11      31684 non-null  category
 14  REC_SPG_INNB_12      31684 non-null  category
 15  REC_SPG_INNB_13    

In [12]:
X_train=train.drop('운송장_건수',axis=1)
y_train=train['운송장_건수']
X_test=test

In [16]:
cat_param={'learning_rate': 0.018272261776066247, 
           'bagging_temperature': 63.512210106407046, 
           'n_estimators': 3794, 
           'max_depth': 11, 
           'random_strength': 15, 
           'colsample_bylevel': 0.49359671220172163, 
           'l2_leaf_reg': 1.7519275289243016e-06, 
           'min_child_samples': 88, 
           'max_bin': 380, 
           'od_type': 'IncToDec'
           }

In [24]:
pip install catboost

     |████████████████████████████████| 76.1 MB 48 kB/s 


In [25]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import random
from catboost import CatBoostRegressor

In [26]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['운송장_건수']):
  folds.append((train_idx,valid_idx))

In [27]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['운송장_건수']):
  folds.append((train_idx,valid_idx))

random.seed(42)
cat_models={}

cat_features =range(X_test.shape[1])

for fold in range(10):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['운송장_건수'],axis=1).iloc[train_idx]
  X_valid = train.drop(['운송장_건수'],axis=1).iloc[valid_idx]
  y_train = train['운송장_건수'][train_idx].values
  y_valid = train['운송장_건수'][valid_idx].values

  cat = CatBoostRegressor(**cat_param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================
0:	learn: 6.7477473	test: 6.7645931	test1: 6.6566720	best: 6.6566720 (0)	total: 403ms	remaining: 25m 30s
100:	learn: 6.6416440	test: 6.6767254	test1: 6.5505933	best: 6.5505933 (100)	total: 12.9s	remaining: 7m 53s
200:	learn: 6.5929003	test: 6.6445401	test1: 6.5214463	best: 6.5214463 (200)	total: 23.5s	remaining: 7m
300:	learn: 6.5303157	test: 6.6089577	test1: 6.4887432	best: 6.4887432 (300)	total: 42.9s	remaining: 8m 18s
400:	learn: 6.5083367	test: 6.5946205	test1: 6.4770041	best: 6.4769531 (398)	total: 53.3s	remaining: 7m 30s
500:	learn: 6.4737775	test: 6.5794197	test1: 6.4600776	best: 6.4600776 (500)	total: 1m 5s	remaining: 7m 13s
600:	learn: 6.3849242	test: 6.5379705	test1: 6.4089630	best: 6.4089630 (600)	total: 1m 19s	remaining: 7m
700:	learn: 6.0644270	test: 6.3824159	test1: 6.1876716	best: 6.1866786 (695)	total: 1m 37s	remaining: 7m 12s
800:	learn: 5.6891302	test: 6.2474066	test1: 6.0554334	best: 6.

In [29]:
sample.loc[:,'운송장_건수']=0
for fold in range(10):
  sample.loc[:,'운송장_건수'] += cat_models[fold].predict(test)/10

In [31]:
sample.loc[sample.운송장_건수>30,'운송장_건수']=sample.loc[sample.운송장_건수>30,'운송장_건수']*4.8
sample.to_csv('sample.csv',index = False)